This notebook checks the inferencing result, and compares it with the dataset we have.

In [21]:
import pandas as pd
from collections import defaultdict
from obspy import UTCDateTime
import numpy as np
from bisect import bisect_left
from pathlib import Path

In [15]:
old_path="/mnt/home/xiziyi/Packages_Research/PhaseNet-TF/dataset/phase_picks.csv"
# old_path="/mnt/scratch/xiziyi/inference/new_20230509/train_bench_0_5/phase_arrivals.csv"
# new_path="/mnt/scratch/xiziyi/inference/train_bench/phase_arrivals.csv"
# new_path="/mnt/scratch/xiziyi/inference/train_bench_low_threshold/phase_arrivals.csv"
new_path="/mnt/scratch/xiziyi/inference/new_20230509/train_bench_0_5/phase_arrivals.csv"
old=pd.read_csv(old_path)
# old=pd.read_csv(old_path,header=None,usecols=[0,1,4,5,6,7],names=["net","sta","phase","offset","time","score"])
new=pd.read_csv(new_path,header=None,usecols=[0,1,4,5,6,7],names=["net","sta","phase","offset","time","score"])

In [16]:
len(old),len(new)

(60307, 131434)

In [17]:
# build the search dataset, dict[sta][phase] as a list
# no F01
old_cache=defaultdict(lambda: defaultdict(list))
for i in range(len(old)):
    row=old.iloc[i]
    old_cache[row.station_id][row.phase_type].append(UTCDateTime(row.phase_time))
for sta in old_cache:
    for phase in old_cache[sta]:
        old_cache[sta][phase].sort()
if "F01" in old_cache:
    old_cache.pop("F01")

# old_cache=defaultdict(lambda: defaultdict(list))
# for i in range(len(old)):
#     row=old.iloc[i]
#     old_cache[row.sta][row.phase].append(UTCDateTime(row.time))
# for sta in old_cache:
#     for phase in old_cache[sta]:
#         old_cache[sta][phase].sort()


new_cache=defaultdict(lambda: defaultdict(list))
for i in range(len(new)):
    row=new.iloc[i]
    new_cache[row.sta][row.phase].append(UTCDateTime(row.time))
for sta in new_cache:
    for phase in new_cache[sta]:
        new_cache[sta][phase].sort()

In [18]:
examples_to_check=[]

def get_confusion_matrix(sta,phase,real,predict,threshold):
    # * now we use the binary search
    # for each real, search in predict
    tp,fn=0,0
    for each in real:
        pos=bisect_left(predict,each)
        case1=False
        if pos>0:
            case1=np.abs(predict[pos-1]-each)<=threshold
        case2=False
        if pos<len(predict):
            case2=np.abs(predict[pos]-each)<=threshold
        if case1 or case2:
            tp+=1
        else:
            fn+=1
            if phase=="S":
                examples_to_check.append((sta,phase,each,predict[pos-1] if pos>0 else None,predict[pos] if pos<len(predict) else None))

    # for each predict, search in real
    tp_recalculate,fp=0,0
    for each in predict:
        pos=bisect_left(real,each)
        case1=False
        if pos>0:
            case1=np.abs(real[pos-1]-each)<=threshold
        case2=False
        if pos<len(real):
            case2=np.abs(real[pos]-each)<=threshold
        if case1 or case2:
            tp_recalculate+=1
        else:
            fp+=1
    try:
        assert tp==tp_recalculate
    except:
        print(f"{sta} {phase} {tp} {tp_recalculate}")
    return tp,fn,fp

In [19]:
threshold=2

tp,fn,fp=defaultdict(int),defaultdict(int),defaultdict(int)
for sta in old_cache:
    for phase in old_cache[sta]:
        x,y,z=get_confusion_matrix(sta,phase,old_cache[sta][phase],new_cache[sta][phase],threshold)
        # print(x,y,z)
        tp[phase]+=x
        fn[phase]+=y
        fp[phase]+=z
tp,fn,fp

A09 S 80 79
A11 S 122 121


(defaultdict(int, {'P': 40288, 'S': 12062, 'PS': 0}),
 defaultdict(int, {'P': 961, 'S': 1872, 'PS': 5124}),
 defaultdict(int, {'P': 53645, 'S': 25434, 'PS': 0}))

In [20]:
examples_to_check

[('A01',
  'S',
  2009-12-02T21:04:24.502960Z,
  2009-12-02T13:55:18.712000Z,
  2009-12-02T21:20:08.932000Z),
 ('A01',
  'S',
  2009-12-04T14:38:40.110170Z,
  2009-12-03T16:37:54.982000Z,
  2009-12-05T05:29:40.012000Z),
 ('A01',
  'S',
  2009-12-05T22:58:51.739110Z,
  2009-12-05T17:53:27.832000Z,
  2009-12-05T23:50:52.962000Z),
 ('A01',
  'S',
  2009-12-06T10:32:18.778000Z,
  2009-12-05T23:50:52.962000Z,
  2009-12-06T10:39:39.292000Z),
 ('A01',
  'S',
  2009-12-06T19:03:59.324320Z,
  2009-12-06T10:39:39.292000Z,
  2009-12-07T02:20:37.962000Z),
 ('A01',
  'S',
  2009-12-10T21:29:49.955340Z,
  2009-12-10T18:41:20.931000Z,
  2009-12-11T18:18:35.621000Z),
 ('A01',
  'S',
  2009-12-16T16:04:51.837460Z,
  2009-12-16T16:00:28.431000Z,
  2009-12-17T10:44:23.601000Z),
 ('A01',
  'S',
  2009-12-17T14:13:33.703800Z,
  2009-12-17T10:44:23.601000Z,
  2009-12-17T16:50:14.201000Z),
 ('A01',
  'S',
  2009-12-17T23:06:40.132920Z,
  2009-12-17T16:50:14.201000Z,
  2009-12-18T12:34:27.959000Z),
 ('A01',
 

In [23]:
# for all examples_to_check third item, check if corresponding file exists in /mnt/scratch/xiziyi/inference/new_20230509/train_bench_0_5
not_generated=[]
for each in examples_to_check:
    sta,phase,time=each[0],each[1],each[2]
    start=UTCDateTime(str(time).split(":")[0])
    end=start+60*60
    # see if /mnt/scratch/xiziyi/inference/new_20230509/train_bench_0_5 has pattern *{sta}{str(start)}.{str(end)}*
    # if not, add to not_generated
    pattern=f"*{sta}.{str(start)}.{str(end)}*"
    if len(list(Path("/mnt/scratch/xiziyi/inference/new_20230509/train_bench_0_5").glob(pattern)))==0:
        not_generated.append(each)

not_generated

KeyboardInterrupt: 

In [25]:
threshold=2

tp,fn,fp=defaultdict(int),defaultdict(int),defaultdict(int)
for sta in old_cache:
    for phase in old_cache[sta]:
        x,y,z=get_confusion_matrix(sta,phase,old_cache[sta][phase],new_cache[sta][phase],threshold)
        # print(x,y,z)
        tp[phase]+=x
        fn[phase]+=y
        fp[phase]+=z
tp,fn,fp

A09 S 80 79
A11 S 122 121


(defaultdict(int, {'P': 40288, 'S': 12062, 'PS': 0}),
 defaultdict(int, {'P': 961, 'S': 1872, 'PS': 5124}),
 defaultdict(int, {'P': 53645, 'S': 25434, 'PS': 0}))

In [26]:
examples_to_check

[('A01',
  'S',
  2009-12-02T21:04:24.502960Z,
  2009-12-02T13:55:18.712000Z,
  2009-12-02T21:20:08.932000Z),
 ('A01',
  'S',
  2009-12-04T14:38:40.110170Z,
  2009-12-03T16:37:54.982000Z,
  2009-12-05T05:29:40.012000Z),
 ('A01',
  'S',
  2009-12-05T22:58:51.739110Z,
  2009-12-05T17:53:27.832000Z,
  2009-12-05T23:50:52.962000Z),
 ('A01',
  'S',
  2009-12-06T10:32:18.778000Z,
  2009-12-05T23:50:52.962000Z,
  2009-12-06T10:39:39.292000Z),
 ('A01',
  'S',
  2009-12-06T19:03:59.324320Z,
  2009-12-06T10:39:39.292000Z,
  2009-12-07T02:20:37.962000Z),
 ('A01',
  'S',
  2009-12-10T21:29:49.955340Z,
  2009-12-10T18:41:20.931000Z,
  2009-12-11T18:18:35.621000Z),
 ('A01',
  'S',
  2009-12-16T16:04:51.837460Z,
  2009-12-16T16:00:28.431000Z,
  2009-12-17T10:44:23.601000Z),
 ('A01',
  'S',
  2009-12-17T14:13:33.703800Z,
  2009-12-17T10:44:23.601000Z,
  2009-12-17T16:50:14.201000Z),
 ('A01',
  'S',
  2009-12-17T23:06:40.132920Z,
  2009-12-17T16:50:14.201000Z,
  2009-12-18T12:34:27.959000Z),
 ('A01',
 

In [18]:
threshold=1.5

tp,fn,fp=defaultdict(int),defaultdict(int),defaultdict(int)
for sta in old_cache:
    for phase in old_cache[sta]:
        x,y,z=get_confusion_matrix(sta,phase,old_cache[sta][phase],new_cache[sta][phase],threshold)
        # print(x,y,z)
        tp[phase]+=x
        fn[phase]+=y
        fp[phase]+=z
{
    "tp":tp,
    "fb":fn,
    "fp":fp
}

A09 S 79 78
A11 S 120 119


{'tp': defaultdict(int, {'P': 40208, 'S': 11951, 'PS': 0}),
 'fb': defaultdict(int, {'P': 1041, 'S': 1983, 'PS': 5124}),
 'fp': defaultdict(int, {'P': 53725, 'S': 25545, 'PS': 0})}

In [19]:
threshold=1

tp,fn,fp=defaultdict(int),defaultdict(int),defaultdict(int)
for sta in old_cache:
    for phase in old_cache[sta]:
        x,y,z=get_confusion_matrix(sta,phase,old_cache[sta][phase],new_cache[sta][phase],threshold)
        # print(x,y,z)
        tp[phase]+=x
        fn[phase]+=y
        fp[phase]+=z
{
    "tp":tp,
    "fn":fn,
    "fp":fp
}

A09 S 77 76
A11 S 116 115


{'tp': defaultdict(int, {'P': 40018, 'S': 11679, 'PS': 0}),
 'fn': defaultdict(int, {'P': 1231, 'S': 2255, 'PS': 5124}),
 'fp': defaultdict(int, {'P': 53915, 'S': 25817, 'PS': 0})}

In [20]:
threshold=0.5

tp,fn,fp=defaultdict(int),defaultdict(int),defaultdict(int)
for sta in old_cache:
    for phase in old_cache[sta]:
        x,y,z=get_confusion_matrix(sta,phase,old_cache[sta][phase],new_cache[sta][phase],threshold)
        # print(x,y,z)
        tp[phase]+=x
        fn[phase]+=y
        fp[phase]+=z
{
    "tp":tp,
    "fn":fn,
    "fp":fp
}

A09 S 57 56
A11 S 100 99


{'tp': defaultdict(int, {'P': 39394, 'S': 10516, 'PS': 0}),
 'fn': defaultdict(int, {'P': 1855, 'S': 3418, 'PS': 5124}),
 'fp': defaultdict(int, {'P': 54539, 'S': 26980, 'PS': 0})}